In [2]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import cvxpy as cp
from math import ceil

data = pd.read_excel('data.xlsx', header=None).values.flatten()

In [3]:
requirements = np.array([102., 102., 103., 103., 104., 104., 105., 105.])
requirements

array([102., 102., 103., 103., 104., 104., 105., 105.])

## 引入问题4算出的初始数据

In [5]:
TOTAL_WEEK = 8
start_bodies = 92
start_hands = 712
start_need_resting_hands = 92 * 4
K = 20
waste_rate = 0.1

In [4]:
working_hands = cp.Parameter(TOTAL_WEEK)
waste_hands = cp.Parameter(TOTAL_WEEK)
working_hands.value = 4 * requirements
waste_hands.value = np.array([
    4 * round(x * waste_rate) if x % 1 != 0.5 else ceil(x * waste_rate) 
    for x in requirements
])

teaching_hands = cp.Variable(TOTAL_WEEK, integer=True)
resting_hands = cp.Variable(TOTAL_WEEK, integer=True)
new_hands = cp.Variable((3, TOTAL_WEEK), integer=True)
new_skill_hands = cp.Variable(integer=True)
# new_tested_bodies = cp.Variable(integer=True)

# 机械手购买的分段点
hand_price = cp.Parameter((3, 1))
hand_price.value = np.array([100, 90, 80]).reshape(3, 1)

hand_y = cp.Variable((3, TOTAL_WEEK), boolean=True)

In [164]:
working_hands.value, waste_hands.value

(array([408., 408., 412., 412., 416., 416., 420., 420.]),
 array([40, 40, 40, 40, 40, 40, 40, 40]))

In [165]:
new_skill_hands.value

In [6]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == start_hands + new_skill_hands,
    resting_hands[0] >= start_need_resting_hands,
    K * teaching_hands >= cp.sum(new_hands, axis=0),

    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0,
    new_skill_hands >= 0,

    20 * hand_y[1] <= new_hands[0],
    new_hands[0] <= 20 * hand_y[0],
    20 * hand_y[2] <= new_hands[1],
    new_hands[1]  <= 20 * hand_y[1],
    new_hands[2] <= 10000 * hand_y[2]
]

In [7]:
for t in range(TOTAL_WEEK - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - waste_hands[t])

# cons.append(resting_hands[1] + working_hands[1] + teaching_hands[1] == \
#     resting_hands[0] + working_hands[0] + cp.sum(new_hands, axis=0)[0] - waste_hands[0]
# )

for t in range(TOTAL_WEEK - 1):
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + cp.sum(new_hands, axis=0)[t] - waste_hands[t]
    )

In [8]:
obj = cp.Minimize(
    cp.sum(cp.multiply(new_hands, hand_price))\
    + 150 * new_skill_hands\
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(teaching_hands)\
    + 10 * cp.sum(new_hands)
)

In [10]:
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CPLEX, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 03 11:49:43 PM: Your problem has 65 variables, 26 constraints, and 19 parameters.
(CVXPY) May 03 11:49:43 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 03 11:49:43 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 03 11:49:43 PM: Compiling problem (target solver=CPLEX).
(CVXPY) May 03 11:49:43 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> CPLEX
(CVXPY) May 03 11:49:43 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 03 11:49:43 PM: Applying reduction Qp2Symboli

Tried aggregator 2 times.
MIP Presolve eliminated 59 rows and 11 columns.
MIP Presolve modified 1 coefficients.
Aggregator did 9 substitutions.
Reduced MIP has 37 rows, 45 columns, and 125 nonzeros.
Reduced MIP has 14 binaries, 31 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.14 ticks)
Found incumbent of value 71180.000000 after 0.00 sec. (0.26 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 37 rows, 45 columns, and 125 nonzeros.
Reduced MIP has 14 binaries, 31 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 7.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.00 sec. (0.12 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Boun

55290.0

In [11]:
working_bodies = cp.Parameter(TOTAL_WEEK)
waste_bodies = cp.Parameter(TOTAL_WEEK)
working_bodies.value = requirements
waste_bodies.value = np.array([
    round(x * waste_rate) if x % 1 != 0.5 else ceil(x * waste_rate) 
    for x in requirements
])

resting_bodies = cp.Variable(TOTAL_WEEK, integer=True)
new_bodies = cp.Variable((3, TOTAL_WEEK), integer=True)
new_tested_bodies = cp.Variable(integer=True)

# 机械手购买的分段点
body_price = cp.Parameter((3, 1))
body_price.value = np.array([200, 180, 160]).reshape(3, 1)

body_y = cp.Variable((3, TOTAL_WEEK), boolean=True)

In [12]:
cons2 = [
    resting_bodies[0] + working_bodies[0] == start_bodies + new_tested_bodies,

    resting_bodies >= 0,
    new_bodies >= 0,
    new_tested_bodies >= 0,

    5 * body_y[1] <= new_bodies[0],
    new_bodies[0] <= 5 * body_y[0],
    5 * body_y[2] <= new_bodies[1],
    new_bodies[1]  <= 5 * body_y[1],
    new_bodies[2] <= 10000 * body_y[2]
]

for t in range(TOTAL_WEEK - 1):
    cons2.append(resting_bodies[t + 1] + working_bodies[t + 1]== \
        resting_bodies[t] + working_bodies[t] + cp.sum(new_bodies, axis=0)[t] - waste_bodies[t]
    )

In [13]:
obj2 = cp.Minimize(
    cp.sum(cp.multiply(new_bodies, body_price))\
    + 10 * cp.sum(resting_bodies) + 300 * new_tested_bodies
)

In [14]:
prob2 = cp.Problem(obj2, cons2)
prob2.solve(solver=cp.CPLEX, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 03 11:51:09 PM: Your problem has 57 variables, 16 constraints, and 19 parameters.
(CVXPY) May 03 11:51:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 03 11:51:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 03 11:51:09 PM: Compiling problem (target solver=CPLEX).
(CVXPY) May 03 11:51:09 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> CPLEX
(CVXPY) May 03 11:51:09 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 03 11:51:09 PM: Applying reduction Qp2Symboli

Tried aggregator 2 times.
MIP Presolve eliminated 43 rows and 11 columns.
Aggregator did 10 substitutions.
Reduced MIP has 28 rows, 36 columns, and 98 nonzeros.
Reduced MIP has 14 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.11 ticks)
Found incumbent of value 17980.000000 after 0.00 sec. (0.29 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 28 rows, 36 columns, and 98 nonzeros.
Reduced MIP has 14 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 7.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.00 sec. (0.07 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0      

16090.00000000001

In [174]:
working_hands.value, resting_hands.value, waste_hands.value

(array([408., 408., 412., 412., 416., 416., 420., 420.]),
 array([298., 368., 417., 372., 421., 376., 420., 380.]),
 array([40, 40, 40, 40, 40, 40, 40, 40]))

In [175]:
hand_y.value

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  0.,  1.,  0., -0.],
       [ 1.,  1.,  0.,  1.,  0.,  1.,  0., -0.]])

In [176]:
new_hands.value, teaching_hands.value, new_tested_bodies.value

(array([[20., 20.,  0., 20.,  0., 20.,  0.,  0.],
        [20., 20.,  0., 20.,  0., 20.,  0.,  0.],
        [75., 48.,  0., 48.,  0., 43.,  0., -0.]]),
 array([6., 5., 0., 5., 0., 5., 0., 0.]),
 array(10.))

## 输出变量

In [15]:
prob.value + prob2.value

71380.00000000001

In [19]:
answers = [
    np.round(np.sum(new_bodies.value, axis=0)),
    np.round(np.sum(new_hands.value, axis=0)),
    np.round(resting_hands.value),
    np.round(resting_bodies.value),
    np.round(np.sum(new_hands.value, axis=0) + teaching_hands.value),
    np.sum(np.round(new_bodies.value) * body_price.value, axis=0) + \
        np.sum(np.round(new_hands.value) * hand_price.value, axis=0) + \
        5 * np.round(resting_hands.value) + \
        10 * np.round(resting_bodies.value) + \
        10 * np.round(np.sum(new_hands.value, axis=0) + teaching_hands.value)
]

In [20]:
np.sum(answers[5]) + 150 * np.round(new_skill_hands.value) + \
        300 * np.round(new_tested_bodies.value)

71380.0

In [21]:
answers = np.array(answers).T

In [181]:
# answers[25]

In [22]:
part_ans = np.array(answers[np.arange(8), :], dtype=np.int32)
part_ans = np.concatenate([part_ans, np.sum(answers, keepdims=True, axis=0)])

In [24]:
df = pd.DataFrame(part_ans, index=list(range(105, 113)) + ['total'],
    # columns=['购买的容器艇数量', '购买的操作手数量', '保养的操作手数量', '保养的容器艇数量', '参与训练的操作手数量', '总成本']
)
df.to_excel('prob5_part1_answer.xlsx')
df

,0,1,2,3,4,5
105,31.0,84.0,368.0,0.0,89.0,15310.0
106,0.0,0.0,417.0,21.0,0.0,2295.0
107,0.0,90.0,368.0,10.0,95.0,10690.0
108,21.0,0.0,423.0,0.0,0.0,5775.0
109,0.0,125.0,372.0,10.0,132.0,13880.0
110,21.0,0.0,464.0,0.0,0.0,5980.0
111,0.0,0.0,420.0,10.0,0.0,2200.0
112,0.0,0.0,380.0,0.0,0.0,1900.0
total,73.0,299.0,3212.0,51.0,316.0,58030.0


In [25]:
58030 + 150 * new_skill_hands.value + 300 * new_tested_bodies.value

71380.0

In [26]:
new_skill_hands.value, new_tested_bodies.value

(array(69.), array(10.))

## 问题五·方案一开头条件